<a href="https://colab.research.google.com/github/amnnsharma/GlidePro/blob/main/Dynacard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.interpolate import interp1d

def fit_curve(x_data, y_data, kind='cubic'):
    interpolated_function = interp1d(x_data, y_data, kind=kind, fill_value="extrapolate")
    return interpolated_function

In [4]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Path to your Excel file in Google Drive
file_path = '/content/drive/My Drive/deviation_data.xlsx'

# Load Excel file into a DataFrame
df = pd.read_excel(file_path)

# Display the loaded DataFrame
df.columns=['SN', 'MD', 'Inclination', 'Azimuth']
df=df.drop(columns=['SN'])
md=df['MD'].astype(float).tolist()
inclination=df['Inclination'].astype(float).tolist()
azimuth=df['Azimuth'].astype(float).tolist()
#print(df)

In [23]:
import numpy as np
import math

T_s=[]

for i in range(len(md)):
  T_s.append(np.array([math.sin(math.radians(inclination[i]))*math.cos(math.radians(azimuth[i])),
                       math.sin(math.radians(inclination[i]))*math.sin(math.radians(azimuth[i])),
                       math.cos(math.radians(inclination[i]))]))


In [35]:
gamma=[]
rc=[]
B_s=[]

for i in range(len(md)-1):
  if T_s[i+1][0]!=T_s[i][0] and T_s[i+1][1]!=T_s[i][1] and T_s[i+1][2]!=T_s[i][2]:
    gamma_i=math.acos(np.dot(T_s[i], T_s[i+1]))
    gamma.append(gamma_i)
    rc.append((md[i+1]-md[i])/gamma_i)
  else:
    gamma.append(0)
    rc.append(np.inf)

  B_s.append(np.cross(T_s[i], T_s[i+1]))


N_s=[]

for i in range(len(md)-1):
  if T_s[i][1]==0:
    N_s.append(np.array([0, 1, 0]))
  else:
    f=(1+(T_s[i][0]/T_s[i][1])**2)**0.5
    N_s.append(np.array([1/f, -T_s[i][0]/T_s[i][1]/f, 0]))


In [38]:
N_s

[array([0, 1, 0]),
 array([0, 1, 0]),
 array([0.42261826, 0.90630779, 0.        ]),
 array([0.64278761, 0.76604444, 0.        ]),
 array([0.61566148, 0.78801075, 0.        ]),
 array([0.48480962, 0.87461971, 0.        ]),
 array([0.27563736, 0.9612617 , 0.        ]),
 array([0.2419219 , 0.97029573, 0.        ]),
 array([0.20791169, 0.9781476 , 0.        ]),
 array([0.190809  , 0.98162718, 0.        ]),
 array([0.17364818, 0.98480775, 0.        ]),
 array([0.15643447, 0.98768834, 0.        ]),
 array([0.08715574, 0.9961947 , 0.        ]),
 array([0.01745241, 0.9998477 , 0.        ]),
 array([ 0.0348995 , -0.99939083,  0.        ]),
 array([ 0.1391731 , -0.99026807,  0.        ]),
 array([ 0.18154904, -0.98338189,  0.        ]),
 array([ 0.190809  , -0.98162718,  0.        ])]